In [29]:
!pip install google-api-python-client oauth2client

In [30]:
import os
import re
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [31]:
SCOPES = ['https://www.googleapis.com/auth/documents']

markdown_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
"""

In [32]:
class MarkdownToGoogleDocs:
    def __init__(self):
        self.service = None

    def authenticate(self):
        """Handle Google Docs API authentication in Colab environment."""
        try:
            auth.authenticate_user()
            creds = GoogleCredentials.get_application_default()
            self.service = build('docs', 'v1', credentials=creds)
            return True
        except Exception as e:
            print(f"Authentication error: {str(e)}")
            return False

    def create_document(self, title):
        """Create a new Google Doc with the specified title."""
        try:
            document = self.service.documents().create(body={'title': title}).execute()
            return document.get('documentId')
        except HttpError as e:
            print(f"Error creating document: {str(e)}")
            return None

    def parse_markdown(self, markdown_text):
        """Parse markdown text and generate Google Docs API requests."""
        requests = []
        current_index = 1

        lines = markdown_text.split('\n')

        for line in lines:
            if line.startswith('#'):
                level = len(re.match(r'^#+', line).group())
                text = line.lstrip('#').strip()
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': text + '\n'
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(text)
                        },
                        'paragraphStyle': {
                            'namedStyleType': f'HEADING_{level}'
                        },
                        'fields': 'namedStyleType'
                    }
                })
                current_index += len(text) + 1
                continue

            if line.strip().startswith(('*', '-')):
                text = line.strip().lstrip('*-').strip()
                indent_level = (len(line) - len(line.lstrip())) // 2

                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': text + '\n'
                    }
                })

                requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(text)
                        },
                        'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
                    }
                })

                if indent_level > 0:
                    requests.append({
                        'updateParagraphStyle': {
                            'range': {
                                'startIndex': current_index,
                                'endIndex': current_index + len(text)
                            },
                            'paragraphStyle': {
                                'indentStart': {
                                    'magnitude': 36 * indent_level,
                                    'unit': 'PT'
                                }
                            },
                            'fields': 'indentStart'
                        }
                    })

                checkbox_match = re.match(r'\[([ x])\]\s*(.+)$', text)
                if checkbox_match:
                    checked = checkbox_match.group(1) == 'x'
                    if checked:
                        requests.append({
                            'updateTextStyle': {
                                'range': {
                                    'startIndex': current_index,
                                    'endIndex': current_index + len(text)
                                },
                                'textStyle': {
                                    'backgroundColor': {
                                        'color': {
                                            'rgbColor': {
                                                'red': 0.9,
                                                'green': 0.9,
                                                'blue': 0.9
                                            }
                                        }
                                    }
                                },
                                'fields': 'backgroundColor'
                            }
                        })

                mentions = re.finditer(r'@(\w+)', text)
                for mention in mentions:
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': current_index + mention.start(),
                                'endIndex': current_index + mention.end()
                            },
                            'textStyle': {
                                'foregroundColor': {
                                    'color': {
                                        'rgbColor': {
                                            'blue': 0.8,
                                            'red': 0.2,
                                            'green': 0.2
                                        }
                                    }
                                },
                                'bold': True
                            },
                            'fields': 'foregroundColor,bold'
                        }
                    })

                current_index += len(text) + 1
                continue

            if line.strip():
                requests.append({
                    'insertText': {
                        'location': {'index': current_index},
                        'text': line.strip() + '\n'
                    }
                })
                current_index += len(line.strip()) + 1

        return requests

    def update_document(self, document_id, markdown_text):
        """Update the Google Doc with formatted content."""
        try:
            requests = self.parse_markdown(markdown_text)
            if requests:
                self.service.documents().batchUpdate(
                    documentId=document_id,
                    body={'requests': requests}
                ).execute()
            return True
        except HttpError as e:
            print(f"Error updating document: {str(e)}")
            return False

In [33]:
def main():
    converter = MarkdownToGoogleDocs()

    if converter.authenticate():
        doc_id = converter.create_document("Product Team Sync")
        if doc_id:
            if converter.update_document(doc_id, markdown_text):
                print(f"Successfully created document: https://docs.google.com/document/d/{doc_id}")
            else:
                print("Failed to update document")
        else:
            print("Failed to create document")
    else:
        print("Authentication failed")

if __name__ == '__main__':
    main()

Successfully created document: https://docs.google.com/document/d/1lxZkMSy_se92KXgy3OdWKUVcu2b-ppbZfiOqkLChUgE
